In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score


In [2]:
# --- 1. Carga y Preprocesamiento ---
def load_and_preprocess_data(filepath):
    # Cargar el archivo parquet especificado en el notebook original
    try:
        df = pd.read_parquet(filepath)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{filepath}'. Asegúrate de que el dataset esté cargado.")
        return None, None

    print("Dataframe Original:")
    print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

    # Limpieza de columnas no útiles (igual que en el original)
    drop_cols = ['Unnamed: 0', 'cc_num', 'first', 'last', 'street', 'city', 'state', 'zip',
                 'trans_num', 'unix_time', 'trans_date_trans_time', 'dob', 'Unnamed: 23', '6006']
    df = df.drop(columns=[c for c in drop_cols if c in df.columns])

    # Codificación de variables categóricas
    le = LabelEncoder()
    for col in ['merchant', 'category', 'gender', 'job']:
        if col in df.columns:
            df[col] = le.fit_transform(df[col].astype(str))

    df = df.fillna(0)

    # --- CAMBIO REALIZADO: Balanceo 1:1 ---
    frauds = df[df['is_fraud'] == 1]

    # Tomamos la misma cantidad de 'no fraudes' que de 'fraudes' (relación 1:1)
    # En el original era: n=len(frauds)*5
    non_frauds = df[df['is_fraud'] == 0].sample(n=len(frauds), random_state=42)

    # Concatenar y mezclar
    df_balanced = pd.concat([frauds, non_frauds]).sample(frac=1, random_state=42)

    X = df_balanced.drop('is_fraud', axis=1)
    y = df_balanced['is_fraud']

    print("\nDataframe Balanceado (1:1):")
    print(f"Total muestras: {len(df_balanced)}")
    print(f"Fraudes: {len(frauds)}")
    print(f"No Fraudes: {len(non_frauds)}")

    # Escalado de datos
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

In [3]:

# --- 2. Ejecución del Modelo (Decision Tree) ---
def run_fraud_detection_model():
    # Nombre del archivo utilizado en el notebook
    filename = "0000.parquet"

    print("--- Cargando datos ---")
    X, y = load_and_preprocess_data(filename)

    if X is None:
        return

    # División de datos (Train/Test) estándar, sin clientes
    print("\nDividiendo datos en Entrenamiento (80%) y Prueba (20%)...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    print("Entrenando modelo Decision Tree...")
    model = DecisionTreeClassifier(max_depth=5, random_state=42)

    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicción
    y_pred = model.predict(X_test)

    # Evaluación
    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)

    print("\n--- Resultados de Detección de Fraude (Decision Tree) ---")
    print(f"Accuracy (Exactitud): {acc:.4f}")
    print(f"Recall (Sensibilidad): {rec:.4f}")
    print(f"ROC-AUC: {auc:.4f}")



In [4]:
# Ejecutar el script
if __name__ == "__main__":
    run_fraud_detection_model()

--- Cargando datos ---
Dataframe Original:
Filas: 1048575, Columnas: 25

Dataframe Balanceado (1:1):
Total muestras: 12012
Fraudes: 6006
No Fraudes: 6006

Dividiendo datos en Entrenamiento (80%) y Prueba (20%)...
Entrenando modelo Decision Tree...

--- Resultados de Detección de Fraude (Decision Tree) ---
Accuracy (Exactitud): 0.9230
Recall (Sensibilidad): 0.9283
ROC-AUC: 0.9229
